# SD212: Graph mining
## Lab 3: PageRank

In this lab, you will learn to compute, use and interpret various [PageRank](https://en.wikipedia.org/wiki/PageRank) scores.

## Import

In [ ]:
from IPython.display import SVG

In [ ]:
import numpy as np
from scipy import sparse

In [ ]:
from sknetwork.data import load_netset, linear_graph, miserables
from sknetwork.linalg import normalize
from sknetwork.ranking import PageRank, BiPageRank, top_k
from sknetwork.visualization import svg_graph

## Data

We will work on the following graphs (see the [NetSet](https://netset.telecom-paris.fr/) collection for details):
* Openflights (graph)
* WikiVitals (digraph)
* Cinema (bigraph)

In [ ]:
openflights = load_netset('openflights')
wikivitals = load_netset('wikivitals')
cinema = load_netset('cinema')

## 1. Graphs

The PageRank corresponds to the stationary distribution of a random walk with restart probability $1-\alpha$. Unless otherwise specified, we take the default value $\alpha = 0.85$ and the restart probability distribution is uniform over the set of nodes.

## Linear graph

Consider a linear graph:

In [ ]:
n = 10

In [ ]:
graph = linear_graph(n, True)
adjacency = graph.adjacency
position = graph.position

In [ ]:
image = svg_graph(adjacency, position, names=np.arange(n))

In [ ]:
SVG(image)

## To do

* What are the two best ranked nodes? Try with different values of $\alpha$ and interpret the results.
* What is the exact PageRank vector when $\alpha=1$? You need to compute it!

In [ ]:
pagerank = PageRank(damping_factor=0.85, solver='lanczos')

In [ ]:
scores = pagerank.fit_transform(adjacency)

In [ ]:
image = svg_graph(adjacency, position, names=np.arange(n), scores=scores)

In [ ]:
SVG(image)

## Les Misérables


In [ ]:
graph = miserables(True)

In [ ]:
adjacency = graph.adjacency
position = graph.position
names = graph.names

In [ ]:
image = svg_graph(adjacency, position, names, scale=2)

In [ ]:
SVG(image)

## To do

* Display the graph with PageRank scores.
* List the 10 best ranked characters.
* Compare with:
    1. the 10 nodes of highest degrees,
    2. the 10 nodes of highest weights,
* Try different values of $\alpha$ (e.g., 0.1, 0.9, 0.99) and interpret the results.

In [ ]:
pagerank = PageRank(damping_factor=0.1, solver='lanczos')

In [ ]:
scores = pagerank.fit_transform(adjacency)

In [ ]:
# scores in log scale are more visible
image = svg_graph(adjacency, position, names, scores=np.log(scores), scale=2)

In [ ]:
SVG(image)

In [ ]:
top_nodes = top_k(scores, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
n = len(names)

In [ ]:
weights = adjacency.dot(np.ones(n))

In [ ]:
top_nodes = top_k(weights, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
degrees = (adjacency > 0).dot(np.ones(n))

In [ ]:
top_nodes = top_k(degrees, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

## Openflights


In [ ]:
graph = openflights

In [ ]:
adjacency = graph.adjacency
position = graph.position
names = graph.names

In [ ]:
image = svg_graph(adjacency, position, width=800, height=400, display_node_weight=True, display_edges=False)

In [ ]:
SVG(image)

## To do

* Display the same world map with PageRank scores.
* List the 10 best ranked airports, and compare with the 10 airports of highest traffic.
* Display the world map with Personalized PageRank scores, starting from Tokyo international airport.
* List the corresponding 10 best ranked airports.

In [ ]:
pagerank = PageRank()

In [ ]:
scores = pagerank.fit_transform(adjacency)

In [ ]:
image = svg_graph(adjacency, position, scores=np.log(scores), node_order=np.argsort(scores), width=800, height=400, display_node_weight=True, display_edges=False)

In [ ]:
SVG(image)

In [ ]:
top_nodes = top_k(scores, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
n = adjacency.shape[0]

In [ ]:
weights = adjacency.dot(np.ones(n))

In [ ]:
top_nodes = top_k(weights, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
{i: name for i, name in enumerate(names) if 'Tokyo' in name}

In [ ]:
tokyo = 1084

In [ ]:
pagerank = PageRank()

In [ ]:
scores = pagerank.fit_transform(adjacency, seeds = {tokyo: 1})

In [ ]:
image = svg_graph(adjacency, position, scores=np.log(scores), node_order=np.argsort(scores), width=800, height=400, display_node_weight=True, display_edges=False) 

In [ ]:
SVG(image)

In [ ]:
top_nodes = top_k(scores, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

## 2. Directed graphs

## Wikipedia Vitals

In [ ]:
graph = wikivitals

In [ ]:
adjacency = graph.adjacency
names = graph.names

## To do

* List the 10 best ranked pages of Wikipedia Vitals.
* Compare with the 10 nodes of highest out-degrees and the 10 nodes of highest in-degrees. Interpret the results.
* Which article of Wikipedia Vitals is in the top-20 of PageRank but not in the top-20 of in-degrees?

In [ ]:
pagerank = PageRank(solver='lanczos')

In [ ]:
scores = pagerank.fit_transform(adjacency)

In [ ]:
top_nodes = top_k(scores, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
n = len(names)

In [ ]:
out_degrees = adjacency.dot(np.ones(n))

In [ ]:
top_nodes = top_k(out_degrees, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
in_degrees = adjacency.T.dot(np.ones(n))

In [ ]:
top_nodes = top_k(in_degrees, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
nodes = list(set(top_k(scores, 20)) - set(top_k(in_degrees, 20)))

In [ ]:
print(names[nodes])

## To do

* List the 20 closest pages to **Picasso** in Wikipedia Vitals. Who is the best ranked painter other than Picasso?
* List the 20 closest pages to both **Cat** and **Dog** in Wikipedia Vitals.
* In both cases, propose a method to get the 20 closest pages **specifically** related to the target(s). <br>
**Hint:** Consider the difference between two PageRank scores.

In [ ]:
picasso = 4936

In [ ]:
pagerank=PageRank()

In [ ]:
scores_ppr = pagerank.fit_transform(adjacency, seeds = {picasso: 1})

In [ ]:
top_nodes = top_k(scores_ppr, 20)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
scores_pr = pagerank.fit_transform(adjacency)

In [ ]:
top_nodes = top_k(scores_ppr - scores_pr, 20)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
cat = 1401

In [ ]:
dog = 1395

In [ ]:
scores_ppr = pagerank.fit_transform(adjacency, seeds = {cat: 1, dog: 1})

In [ ]:
top_nodes = top_k(scores_ppr, 20)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
top_nodes = top_k(scores_ppr - scores_pr, 20)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

## 3. Bipartite graphs

## Cinema

In [ ]:
graph = cinema

In [ ]:
biadjacency = graph.biadjacency
movies = graph.names_row
actors = graph.names_col

## To do

* List the top-10 movies in terms of PageRank.
* List the 5 closest movies to **Mon Oncle**. 
* List the 5 closest actors to **Catherine Deneuve**.

In [ ]:
pagerank = BiPageRank()

In [ ]:
scores = pagerank.fit_transform(biadjacency)

In [ ]:
top_nodes = top_k(scores, 10)

In [ ]:
movies[top_nodes]

In [ ]:
deneuve = 2173

In [ ]:
scores = pagerank.fit_transform(biadjacency, seeds_col = {deneuve:1})

In [ ]:
scores_col = pagerank.scores_col_

In [ ]:
top_nodes = top_k(scores_col, 5)

In [ ]:
actors[top_nodes]

## 4. Directed graphs as bipartite graphs

Directed graphs can be represented as bipartite graphs by duplicating each node, one as source of edges and the other as destination of edges. The biadjacency matrix of the bipartite graph is simply the adjacency matrix of the directed graph. 

The PageRank scores obtained with the bipartite graph differ from those obtained with the directed graph: they correspond to the **forward-backward** random walk in the directed graph, edges being alternately followed in forward and backward directions.

## Wikipedia Vitals

In [ ]:
graph = wikivitals

In [ ]:
adjacency = graph.adjacency
names = graph.names

## To do

Do the same experiments as above using BiPageRank:
* List the 10 best ranked pages of Wikipedia Vitals. Give 2 rankings: one for rows, the other for columns.
* Compare with the 10 nodes of highest out-degrees and the 10 nodes of highest in-degrees, respectively.<br> Interpret the results.

In [ ]:
pagerank = BiPageRank(damping_factor=0.85)

In [ ]:
scores = pagerank.fit_transform(adjacency)

In [ ]:
top_nodes = top_k(scores, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
top_nodes_out = top_k(out_degrees, 10)

In [ ]:
for i, node in enumerate(top_nodes_out):
    print(i + 1, names[node])

In [ ]:
len(set(top_nodes) & set(top_nodes_out))

In [ ]:
top_nodes = top_k(pagerank.scores_col_, 10)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
top_nodes_in = top_k(in_degrees, 10)

In [ ]:
for i, node in enumerate(top_nodes_in):
    print(i + 1, names[node])

In [ ]:
len(set(top_nodes) & set(top_nodes_in))

## To do

Using BiPageRank:
* List the 20 closest pages to **Picasso** in Wikipedia Vitals. 
* List the 20 closest pages to both **Cat** and **Dog** in Wikipedia Vitals.

In your opinion, what is the best algorithm for content recommendation, PageRank or BiPageRank?<br>
Justify your answer.

In [ ]:
pagerank = BiPageRank()

In [ ]:
scores_pr = pagerank.fit_transform(adjacency)

In [ ]:
scores = pagerank.fit_transform(adjacency, seeds_row = {picasso:1})

In [ ]:
top_nodes = top_k(scores, 20)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])

In [ ]:
scores = pagerank.fit_transform(adjacency, seeds_row = {cat:1, dog:1})

In [ ]:
top_nodes = top_k(scores, 20)

In [ ]:
for i, node in enumerate(top_nodes):
    print(i + 1, names[node])